In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import logging

In [2]:
import pickle
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas

In [3]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
data = pd.read_csv('../data2/data_1219.txt', sep='\t')
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,-0.122170,4.0,3,22,0.021705,0.0,NaN,NaN,NaN,...,0.000000,0.000369,1.000000,0.960571,0.022114,0.371930,0.218308,0.062702,1.0,1
1,3862,-0.015598,4.0,3,15,-0.182562,0.0,NaN,NaN,NaN,...,0.000004,0.000036,1.000000,0.960571,0.024451,0.111552,0.444739,0.002961,1.0,5
2,3867,0.000000,3.0,3,4,0.070942,0.0,32.0,13.0,9.53125,...,0.000022,0.001441,1.000000,0.290809,0.006221,0.034154,0.402727,0.044313,1.0,3
3,3861,-0.302475,5.0,2,20,-0.191518,0.0,3.0,234.0,222.33333,...,0.000011,0.000018,0.000000,1.000000,0.024451,0.111552,0.444739,0.002961,1.0,4
4,3862,-0.095008,4.0,1,8,-0.176807,1.0,NaN,NaN,NaN,...,0.000043,0.000027,0.008695,0.960571,0.005739,0.007515,0.438648,0.125989,1.0,5


In [6]:
feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [7]:
train_label=2593669

In [8]:
data = data.fillna(0)
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,-0.122170,4.0,3,22,0.021705,0.0,0.0,0.0,0.00000,...,0.000000,0.000369,1.000000,0.960571,0.022114,0.371930,0.218308,0.062702,1.0,1
1,3862,-0.015598,4.0,3,15,-0.182562,0.0,0.0,0.0,0.00000,...,0.000004,0.000036,1.000000,0.960571,0.024451,0.111552,0.444739,0.002961,1.0,5
2,3867,0.000000,3.0,3,4,0.070942,0.0,32.0,13.0,9.53125,...,0.000022,0.001441,1.000000,0.290809,0.006221,0.034154,0.402727,0.044313,1.0,3
3,3861,-0.302475,5.0,2,20,-0.191518,0.0,3.0,234.0,222.33333,...,0.000011,0.000018,0.000000,1.000000,0.024451,0.111552,0.444739,0.002961,1.0,4
4,3862,-0.095008,4.0,1,8,-0.176807,1.0,0.0,0.0,0.00000,...,0.000043,0.000027,0.008695,0.960571,0.005739,0.007515,0.438648,0.125989,1.0,5


In [11]:
# target编码
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:train_label][feature_cols]
y_train_all = data.iloc[:train_label]['label']
# test = data.iloc[len(train_label):]
# # del data
# assert len(test) == sub_size

# logging.info("train shape %s", train_label.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2,min_samples_split=20,min_samples_leaf=8,max_leaf_nodes=20),
                        algorithm="SAMME.R",n_estimators=50,learning_rate=0.5)
bdt.fit(X_train,y_train)
print("Score_train: ",bdt.score(X_train,y_train))
print("Score_val: ",bdt.score(X_val,y_val))
# model_xgb = xgb.XGBClassifier(n_estimators=3000, n_jobs=-1,max_depth=5, objective='binary:logistic', seed=1000, silent=True)
# model_xgb.fit(X_train, y_train,
#               eval_metric=['logloss', 'auc'],
#               eval_set=[(X_val, y_val)],
#               early_stopping_rounds=50)

[2019-12-20 12:08:05,122] INFO in <ipython-input-11-b932bafc5a86>: feature size 128


Score_train:  0.8498598027696302
Score_val:  0.8493951632336356
